# Sequential

In [15]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [16]:
def get_sequential_model(input_shape):
    model = keras.Sequential(
        [
            # Input
            layers.Input(input_shape),
            
            # 1st Conv block
            layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),
            layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),
            layers.MaxPool2D(),
            layers.BatchNormalization(),
            layers.Dropout(0.5),
            
            # 2nd Conv block
            layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),
            layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),
            layers.MaxPool2D(),
            layers.BatchNormalization(),
            layers.Dropout(0.3),
            
            # Classifier
            layers.GlobalMaxPool2D(),
            layers.Dense(128, activation='relu'),
            layers.Dense(1, activation='sigmoid')
        ]
    )
    return model

input_shape = (256, 256, 3)
model = get_sequential_model(input_shape)

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 256, 256, 64)      1792      
                                                                 
 conv2d_17 (Conv2D)          (None, 256, 256, 64)      36928     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 128, 128, 64)     0         
 2D)                                                             
                                                                 
 batch_normalization_8 (Batc  (None, 128, 128, 64)     256       
 hNormalization)                                                 
                                                                 
 dropout_8 (Dropout)         (None, 128, 128, 64)      0         
                                                                 
 conv2d_18 (Conv2D)          (None, 128, 128, 128)    

# Functional

In [20]:
def get_functional_model(input_shape):
    inputs = keras.Input(input_shape)

    # 1st Conv block
    x = layers.Conv2D(64, 3, strides=1, activation='relu', padding='same')(inputs)
    x = layers.Conv2D(64, 3, strides=1, activation='relu', padding='same')(x)
    x = layers.MaxPool2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)

    # 2nd Conv block
    x = layers.Conv2D(128, 3, strides=1, activation='relu', padding='same')(x)
    x = layers.Conv2D(128, 3, strides=1, activation='relu', padding='same')(x)
    x = layers.MaxPool2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)

    # Classifier
    x = layers.GlobalMaxPool2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    model = keras.Model(inputs, outputs)
    
    return model
    
    
input_shape = (256, 256, 3)
model = get_functional_model(input_shape)
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 conv2d_28 (Conv2D)          (None, 256, 256, 64)      1792      
                                                                 
 conv2d_29 (Conv2D)          (None, 256, 256, 64)      36928     
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 128, 128, 64)     0         
 g2D)                                                            
                                                                 
 batch_normalization_14 (Bat  (None, 128, 128, 64)     256       
 chNormalization)                                                
                                                                 
 dropout_14 (Dropout)        (None, 128, 128, 64)      0   

# Model subclassing

In [22]:
class SimpleCNN(keras.Model):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        self.conv_block1 = keras.Sequential(
            [
                layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),
                layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),
                layers.MaxPool2D(),
                layers.BatchNormalization(),
                layers.Dropout(0.5),
            ], name='conv_block1'
        )
        
        self.conv_block2 = keras.Sequential(
            [
                layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),
                layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),
                layers.MaxPool2D(),
                layers.BatchNormalization(),
                layers.Dropout(0.3),
            ], name='conv_block2'
        )
        self.classifier = keras.Sequential(
            [
                layers.GlobalMaxPool2D(),
                layers.Dense(128, activation='relu'),
                layers.Dense(1, activation='sigmoid')
            ], name='classifier'
        )
        
    def call(self, input_tensor, training=False):
        x = self.conv_block1(input_tensor)
        x = self.conv_block2(x)
        x = self.classifier(x)
        
        return x

In [24]:
input_shape = (None, 256, 256, 3)
model = SimpleCNN()
model.build(input_shape)

model.summary()

Model: "simple_cnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_block1 (Sequential)    (None, 128, 128, 64)      38976     
                                                                 
 conv_block2 (Sequential)    (None, 64, 64, 128)       221952    
                                                                 
 classifier (Sequential)     (None, 1)                 16641     
                                                                 
Total params: 277,569
Trainable params: 277,185
Non-trainable params: 384
_________________________________________________________________


# Model compiling

In [29]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics='accuracy'
)